In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
# Sample data
sentences = ["The quick brown fox jumps over the lazy dog.",
             "I love TensorFlow 2."]
# Tokenize and pad the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
vocab_size = len(word_index) + 1

# Convert sentences to sequences
sequences = tokenizer.texts_to_sequences(sentences)
vocab_size

13

In [8]:
sequences

[[1, 2, 3, 4, 5, 6, 1, 7, 8], [9, 10, 11, 12]]

In [9]:

# Generate training data using skipgrams
word_targets = []
word_contexts = []
labels_list = []

for sequence in sequences:
    pairs, label = skipgrams(sequence, vocabulary_size=vocab_size, window_size=2)
    word_target, word_context = zip(*pairs)

    word_targets.extend(word_target)
    word_contexts.extend(word_context)
    labels_list.extend(label)

In [14]:
# Define the Word2Vec model
embedding_dim = 128

input_word = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

word_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(input_word)
context_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(input_context)

dot_product = tf.keras.layers.Dot(axes=2)([word_embedding, context_embedding])


In [15]:
flatten = tf.keras.layers.Flatten()(dot_product)
output = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)

model = tf.keras.Model(inputs=[input_word, input_context], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Convert word_targets, word_contexts, and labels_list to numpy arrays
word_targets = np.array(word_targets)
word_contexts = np.array(word_contexts)
labels_list = np.array(labels_list)

# Train the model
model.fit(x=[word_targets, word_contexts], y=labels_list, epochs=100, batch_size=512)

# Get the word embeddings
embeddings = model.layers[2].get_weights()[0]

# Function to find similar words
def find_similar_words(word, embeddings, word_index, reverse_word_index, top_n=5):
    if word not in word_index:
        return []

    word_vector = embeddings[word_index[word]]
    similarity = np.dot(embeddings, word_vector) / (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(word_vector))
    sorted_indexes = np.argsort(similarity)[::-1]

    closest_words = [reverse_word_index[i] for i in sorted_indexes[1:top_n+1]]
    return closest_words

# Using the function
similar_to_fox = find_similar_words("quick", embeddings, word_index, reverse_word_index, top_n=5)
print(f"Words similar to 'quick': {similar_to_fox}")


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 0.6937 - accuracy: 0.4750
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6922 - accuracy: 0.6125
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6908 - accuracy: 0.6500
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6893 - accuracy: 0.7000
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6879 - accuracy: 0.7625
Epoch 6/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6864 - accuracy: 0.8000
Epoch 7/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6849 - accuracy: 0.8250
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6834 - accuracy: 0.8625
Epoch 9/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6819 - accuracy: 0.8625
Epoch 10/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6803 - accuracy: 0.8625
Epoch 11/100
1/1 [===

In [16]:
sentences = [
    "Rust is a systems programming language.",
    "It offers memory safety, concurrency, and performance.",
    "Rust achieves memory safety without garbage collection.",
    "It's designed to be fast and reliable.",
    "Cargo is Rust's build system and package manager.",
    "Rust has an ownership system.",
    "The Rust community values inclusive code.",
    "Rust's rich type system and ownership model guarantee memory safety and thread safety.",
    "Rust has a zero-cost abstraction.",
    "Match expressions are a central feature of Rust.",
    "The borrow checker ensures safe memory access in Rust.",
    "Enums in Rust can have data associated with them.",
    "The Rust documentation is comprehensive.",
    "Rust can integrate with other languages easily.",
    "Concurrency in Rust is achieved through threads and async code.",
    "Rust is becoming popular in web development with frameworks like Rocket and Actix."
]


In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams

# Tokenize and pad the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
vocab_size = len(word_index) + 1

# Convert sentences to sequences
sequences = tokenizer.texts_to_sequences(sentences)

# Generate training data using skipgrams
word_targets = []
word_contexts = []
labels_list = []

for sequence in sequences:
    pairs, label = skipgrams(sequence, vocabulary_size=vocab_size, window_size=3)
    word_target, word_context = zip(*pairs)

    word_targets.extend(word_target)
    word_contexts.extend(word_context)
    labels_list.extend(label)

# Define the Word2Vec model
embedding_dim = 128

input_word = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

word_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(input_word)
context_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(input_context)

dot_product = tf.keras.layers.Dot(axes=2)([word_embedding, context_embedding])
flatten = tf.keras.layers.Flatten()(dot_product)
output = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)

model = tf.keras.Model(inputs=[input_word, input_context], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Convert word_targets, word_contexts, and labels_list to numpy arrays
word_targets = np.array(word_targets)
word_contexts = np.array(word_contexts)
labels_list = np.array(labels_list)

# Train the model
model.fit(x=[word_targets, word_contexts], y=labels_list, epochs=200, batch_size=512)

# Get the word embeddings
embeddings = model.layers[2].get_weights()[0]

# Function to find similar words
def find_similar_words(word, embeddings, word_index, reverse_word_index, top_n=5):
    if word not in word_index:
        return []

    word_vector = embeddings[word_index[word]]
    similarity = np.dot(embeddings, word_vector) / (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(word_vector))
    sorted_indexes = np.argsort(similarity)[::-1]

    closest_words = [reverse_word_index[i] for i in sorted_indexes[1:top_n+1]]
    return closest_words

# Using the function
similar_to_rust = find_similar_words("rust", embeddings, word_index, reverse_word_index, top_n=5)
print(f"Words similar to 'rust': {similar_to_rust}")


Epoch 1/200
3/3 [==============================] - 2s 274ms/step - loss: 0.6936 - accuracy: 0.4876
Epoch 2/200
3/3 [==============================] - 0s 150ms/step - loss: 0.6913 - accuracy: 0.6064
Epoch 3/200
3/3 [==============================] - 0s 94ms/step - loss: 0.6895 - accuracy: 0.6968
Epoch 4/200
3/3 [==============================] - 0s 94ms/step - loss: 0.6878 - accuracy: 0.7660
Epoch 5/200
3/3 [==============================] - 0s 159ms/step - loss: 0.6861 - accuracy: 0.8191
Epoch 6/200
3/3 [==============================] - 0s 159ms/step - loss: 0.6844 - accuracy: 0.8528
Epoch 7/200
3/3 [==============================] - 0s 91ms/step - loss: 0.6825 - accuracy: 0.8741
Epoch 8/200
3/3 [==============================] - 0s 176ms/step - loss: 0.6807 - accuracy: 0.8910
Epoch 9/200
3/3 [==============================] - 0s 156ms/step - loss: 0.6786 - accuracy: 0.9034
Epoch 10/200
3/3 [==============================] - 0s 6ms/step - loss: 0.6765 - accuracy: 0.9158
Epoch 11/200
3

These results illustrate how the trained Word2Vec model has learned to relate words based on their context within the provided dataset.

"documentation": This might be due to discussions or sentences that mention Rust in the context of its documentation.

"the": Given that "the" is a common word, it often appears near many other words, including "Rust". It's common for very frequent words to be picked up as similar, especially with smaller datasets.

"becoming": Possibly associated with Rust's growing popularity or trends.

"without": Could be related to the context where Rust is discussed, especially in terms of its features like achieving memory safety "without" garbage collection.

"code": Rust is a programming language, so it's intuitive to find "code" as a similar word.

When using Word2Vec, the results will be significantly influenced by the size and quality of the training data. For more meaningful and context-rich embeddings, a much larger corpus that thoroughly discusses Rust programming would be beneficial.

If you're looking for highly accurate and meaningful embeddings, consider using pretrained embeddings (e.g., from FastText or GloVe) or training on a vast corpus. You can also adjust hyperparameters like window size, embedding dimension, or use advanced architectures like FastText to capture sub-word information.







In [18]:
# Using the function
similar_to_rust = find_similar_words("abstraction", embeddings, word_index, reverse_word_index, top_n=5)
print(f"Words similar to 'abstraction': {similar_to_rust}")

Words similar to 'abstraction': ['expressions', 'zero', 'cost', 'has', 'values']
